In [1]:
import os
from pathlib import Path
import random

import puppets_data
import ConnectionGraphX
import GraphX

import numpy as np
import pandas as pd
import scipy
import networkx as nx

from pyLDLE2 import datasets
from pyLDLE2 import buml_
from pyLDLE2 import util_

from proxi.algorithms.knng import get_knn_graph
from proxi.utils.misc import save_graph, save_weighted_graph
from proxi.utils.process import *
from proxi.utils.distance import abs_correlation

matplotlib.get_backend() =  module://matplotlib_inline.backend_inline


In [14]:
p = "/home/sawyerjack/IdeaProjects/data-git-ignore/PuppetsData"
save_dir_root = '/home/sawyerjack/IdeaProjects/connection-graph-wasserstein/data'

## Just for time and sanity, using n = 100 while I test things.
N_IMAGES = 1000

In [3]:
Y, labelsMat, _ = puppets_data.puppets_data(p)

X.shape =  (8100, 100)


In [15]:
X = np.array(Y.copy())
totalImages = X.shape[0]
sampleIndices = random.sample(range(totalImages), N_IMAGES)
X = [ X[i] for i in sampleIndices ]

X = np.array(X)
X.shape

(1000, 100)

In [16]:
buml_obj = buml_.BUML(local_opts={'algo':'LPCA', 'k': 28},
                      vis_opts={'c': labelsMat[:,0], 'save_dir': save_dir_root},
                      verbose=True, debug=True, exit_at='local_views')

buml_obj.fit(X=X)


local_opts['k_nn0'] = 700 is created.
Options provided:
local_opts:
{
    "Atilde_method": "LDLE_1",
    "N": 100,
    "U_method": "k_nn",
    "algo": "LPCA",
    "alpha": 1,
    "debug": true,
    "delta": 0.9,
    "gl_type": "unnorm",
    "k": 28,
    "k_nn": 49,
    "k_nn0": 700,
    "k_tune": 7,
    "lambda1_decay": 0.75,
    "lambda1_init": 8,
    "lambda1_min": 0.001,
    "max_iter": 300,
    "max_sparsity": 0.9,
    "metric": "euclidean",
    "n_proc": 6,
    "p": 0.99,
    "power": 5,
    "pp_n_thresh": 32,
    "radius": 0.5,
    "reg": 0.0,
    "scale_by": "gamma",
    "tau": 50,
    "to_postprocess": true,
    "verbose": true
}
intermed_opts:
{
    "algo": "best",
    "debug": true,
    "eta_max": 25,
    "eta_min": 5,
    "len_S_thresh": 256,
    "local_algo": "LPCA",
    "metric": "euclidean",
    "n_proc": 6,
    "n_times": 4,
    "verbose": true
}
global_opts:
{
    "add_dim": false,
    "align_transform": "rigid",
    "align_w_parent_only": true,
    "alpha": 0.3,
    "b

In [ ]:
for i in range(N_IMAGES):
    n_i = buml_obj.LocalViews.U[i,:].nonzero()[1]
    for j in [j in n_i if j > i]:




In [ ]:
p = "/home/sawyerjack/IdeaProjects/data-git-ignore/PuppetsData"
save_dir_root = '/home/sawyerjack/IdeaProjects/connection-graph-wasserstein/data'

## Just for time and sanity, using n = 100 while I test things.
N_IMAGES = 1000

In [ ]:
Y, labelsMat, _ = puppets_data.puppets_data(p)

In [ ]:
X = np.array(Y.copy())
totalImages = X.shape[0]
sampleIndices = random.sample(range(totalImages), N_IMAGES)
X = [ X[i] for i in sampleIndices ]

X = np.array(X)
X.shape

In [ ]:
buml_obj = buml_.BUML(local_opts={'algo':'LPCA', 'k': 28},
                      vis_opts={'c': labelsMat[:,0], 'save_dir': save_dir_root},
                      verbose=True, debug=True, exit_at='local_views')

buml_obj.fit(X=X)


In [17]:
for i in range(N_IMAGES):
    n_i = buml_obj.LocalViews.U[i,:].nonzero()[1]
    for j in [j for j in n_i if j > i]:
        n_ij = buml_obj.LocalViews.U[i,:].multiply(buml_obj.LocalViews.U[j,:]).nonzero()[1]
        X_Uij_i = buml_obj.LocalViews.local_param_post.eval_({'view_index': i, 'data_mask': n_ij})
        X_Uij_j = buml_obj.LocalViews.local_param_post.eval_({'view_index': j, 'data_mask': n_ij})
        Tij, _ = util_.custom_procrustes(X_Uij_i, X_Uij_j)

25
24
21
12
15
19
20
18
15
25
21
17
14
23
16
18
19
16
18
17
27
15
19
27
19
17
19
12
10
10
17
10
8
11
11
10
10
9
13
11
21
6
13
15
14
12
10
11
12
6
9
8
10
12
13
25
19
17
20
19
15
12
17
10
20
20
18
23
12
15
21
15
14
14
22
9
15
23
14
16
18
24
16
18
10
26
20
13
22
23
16
22
14
9
26
20
21
13
16
16
21
11
22
21
12
16
13
24
17
13
10
20
18
19
11
21
16
8
7
16
9
26
16
13
14
12
16
14
12
15
17
17
16
19
13
16
16
15
15
17
11
18
16
10
16
16
13
13
23
19
12
2
17
13
17
26
23
14
9
24
12
13
6
17
16
15
15
12
11
16
14
6
15
3
1
22
14
8
13
14
7
13
12
18
13
12
8
9
19
21
24
17
15
16
18
19
19
23
24
18
16
24
6
12
14
15
16
18
20
17
11
19
16
20
8
21
7
14
18
14
10
17
14
8
13
13
10
17
19
21
19
19
18
16
22
19
1
16
17
8
21
24
21
16
20
7
18
18
13
14
18
11
6
25
7
19
18
21
20
19
13
21
11
17
15
7
24
21
8
14
25
3
10
15
14
8
1
23
14
9
14
9
14
12
7
3
9
5
13
8
11
9
21
5
11
26
11
23
13
19
16
22
17
17
11
18
26
23
21
24
11
13
23
19
20
20
14
15
21
23
24
19
12
16
23
21
23
16
10
15
14
27
16
18
21
11
14
18
8
10
15
11
19
18
20
9
25
16
20